In [1]:
import configparser, os, sys

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

sys.path.insert(0, "/home/pleroy/DEV/DIADEM")

import toolsdiadem.tools as dia

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [3]:
dir_E = "/home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/E plane"
dir_H = "/home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/H plane"
root_dir = "/home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/"

In [4]:
Nf, Nstep = dia.get_Nf_Nstep(dir_E)

data_dir: /home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/E plane
Nf = 26, Nstep = 2


In [5]:
f = dia.getF(Nf, "EvEh", dir_E)

data_dir: /home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/E plane
Nf = 26, fmin = 8.0, fmax = 13.0


In [6]:
stepE = dia.getStep(Nstep, "EvEh", dir_E)
stepH = dia.getStep(Nstep, "EvEh", dir_H)

data_dir: /home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/E plane
Nstep 2 (polar), min 0.0, max 90.0
data_dir: /home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/H plane
Nstep 2 (polar), min 0.0, max 90.0


In [7]:
filename = "/home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/E plane/EvEh_000_000.amp"
dia.get_headerSize(filename)

16

In [8]:
prefix_E_plane = "/home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/E plane/EvEh"
prefix_H_plane = "/home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/H plane/EvEh"

In [9]:
scan0, val0 = dia.get_scan_val(prefix_E_plane, 0, 0, ".amp")
scanAxis, stepAxis, stepPositiond = dia.get_axes(filename)

[.amp] Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 0.0


In [10]:
Nscan = scan0.shape[0]

In [26]:
amp_E_plane, pha_E_plane, freq = dia.getAmpPhaArrays(Nscan, Nstep, 10, prefix_E_plane)

[.amp] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 0.0
[.pha] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 0.0
[.amp] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 90.0
[.pha] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 90.0


In [27]:
amp_H_plane, pha_H_plane, freq  = dia.getAmpPhaArrays(Nscan, Nstep, 10, prefix_H_plane)

[.amp] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 0.0
[.pha] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 0.0
[.amp] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 90.0
[.pha] 10.0 GHz, Nscan 361 (azimuth), min -90.0, max 90.0, StepAxis polar, StepPosition 90.0


In [35]:
step = 0
dia.saveData(root_dir, (amp_E_plane, pha_E_plane), scan0, freq, step)

save /home/pleroy/Documents/IETR/PoSAR-MC/X_band/DATASHEETS/ANT/Diagramme_antennes/bande X/10.0_step0_amp_pha_32GHz.data


In [36]:
maxE = np.amax(amp_E_plane)
maxE_3dB = maxE - 3
maxH = np.amax(amp_H_plane)
maxH_3dB = maxH - 3

In [37]:
fig, ax = plt.subplots(1, 1)
ax.plot(scan0, amp_E_plane[0, :], label=f"E plane, {stepAxis} {stepE[0]}")
ax.plot(scan0, amp_E_plane[1, :], label=f"E plane, {stepAxis} {stepE[1]}")
ax.axhline(maxE_3dB, ls='-.', c='orange')
ax.plot(scan0, amp_H_plane[0, :], label=f"H plane, {stepAxis} {stepH[0]}")
ax.plot(scan0, amp_H_plane[1, :], label=f"H plane, {stepAxis} {stepH[1]}")
ax.set_xlabel(scanAxis)
ax.legend()
ax.grid()

In [38]:
def get_aperture_3dB(angle, dB):
    maxdB = np.amax(dB)
    _3dB = maxdB - 3
    idx = np.where(dB - _3dB > 0)[0]
    return angle[idx[-1]] - angle[idx[0]]

In [39]:
aperture_E = get_aperture_3dB(scan0, amp_E_plane[1, :])
aperture_H = get_aperture_3dB(scan0, amp_H_plane[0, :])

In [44]:
fig, ax = plt.subplots(1, 1)
ax.plot(scan0, amp_E_plane[1, :], label=f"polarisation verticale / élévation (plan E)\nouverture {aperture_E}°")
ax.axhline(maxE_3dB, ls='-.', c='C0')
ax.set_xlabel(f"{scanAxis} [°]")
ax.set_ylabel("[dB]")
ax.set_title(f"Flann 16240-20-02, {freq} GHz")
ax.legend()
ax.grid()

In [45]:
fig, ax = plt.subplots(1, 1)
ax.plot(scan0, amp_H_plane[0, :], label=f"polarisation verticale / azimut (plan H)\nouverture {aperture_H}°")
ax.axhline(maxH_3dB, ls='-.', c='C0')
ax.set_xlabel(f"{scanAxis} [°]")
ax.set_ylabel("[dB]")
ax.set_title(f"Flann 16240-20-02, {freq} GHz")
ax.legend()
ax.grid()